# kernel

> IPythonKernel based on pystata

The latest documentation for implementing a wrapper kernel is [here](https://jupyter-client.readthedocs.io/en/latest/wrapperkernels.html), but the current code deviates from those instructions, instead inheriting from the IPython kernel implementation, [IPythonKernel](https://github.com/ipython/ipykernel/blob/main/ipykernel/ipkernel.py).

In [ ]:
#| default_exp kernel
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import test_eq
from fastcore.basics import patch_to

In [ ]:
#| export
from ipykernel.ipkernel import IPythonKernel
from nbstata.config import get_config, launch_stata
from nbstata import parsers
import os
import sys
from packaging import version

In [ ]:
#| export
class PyStataKernel(IPythonKernel):
    implementation = 'nbstata'
    implementation_version = '0.0.1'
    language = 'stata'
    language_version = '17'
    language_info = {
        'name': 'stata',
        'mimetype': 'text/x-stata',
        'codemirror_mode': 'stata',
        'file_extension': '.do',
    }
    banner = "nbstata: a Jupyter kernel for Stata based on pystata"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.stata_ready = False
        self.shell.execution_count = 0
        self.magic_handler = None
        self.env = None

In [ ]:
#| export
@patch_to(PyStataKernel)
def init_stata(self):
    def _set_graph_format(graph_format):
        if graph_format == 'nbstata':
            pass
        else:
            from pystata.config import set_graph_format
            set_graph_format(graph_format)
    
    self.env = get_config()
    if self.env['echo'] not in ('True', 'False', 'None'):
        raise OSError("'" + self.env['echo'] + "' is not an acceptable value for 'echo'.")

    launch_stata(self.env['stata_dir'], self.env['edition'],
                 False if self.env['splash']=='False' else True)

    _set_graph_format(self.env['graph_format'])

    from .magics import StataMagics
    self.magic_handler = StataMagics()

    self.stata_ready = True

In [ ]:
class Cell:
    def __init__(self, kernel, code_w_magics):
        if kernel.env['echo'] == 'None':
            self.noecho = True
            self.echo = False
        elif kernel.env['echo'] == 'True':
            self.noecho = False
            self.echo = True
        else:
            self.noecho = False
            self.echo = False
        self.quietly = False
        self.code = self.magic_handler.magic(code_w_magics, kernel, cell)
    
    def run(self):
        # Execute Stata code after magics
        if self.code != '':
            # Supress echo?
            if self.noecho and not self.quietly:
                noecho_run(code)
            else:
                from pystata.stata import run
                run(code, quietly=self.quietly, inline=True, echo=self.echo)

In [ ]:
def noecho_run(code):
    """
    Split code into program and non-program blocks, running each block noecho
    """
    from pystata.stata import run
    def _run_as_program(clean_non_prog_code):
        _program_name = "temp_nbstata_kernel_program_name"
        _program_define_code = f"program {_program_name}\n{clean_non_prog_code}\nend\n"
        run(_program_define_code, quietly=True)
        run(_program_name, quietly=False, inline=True, echo=False)
        run(f"program drop {_program_name}", quietly=True)

    def _run_non_prog_lines(co):
        clean_non_prog_code = '\n'.join(co)
        if len(co) == 1:  # to avoid outputting extra blank lines
            run(clean_non_prog_code, quietly=False, inline=True, echo=False)
        else:
            _run_as_program(clean_non_prog_code)

    def _run_prog_lines(co):        
        clean_prog_code = '\n'.join(co)
        if co[0] in ['mata', 'mata:']:  # b/c 'quietly' blocks all mata output
            run(clean_prog_code, quietly=False, inline=True, echo=False)
        else:
            run(clean_prog_code, quietly=True, inline=True, echo=False)
            
    cl = parsers.clean_code(code).splitlines()
    co = []
    for c in cl:
        cs = c.strip()

        # Are we starting a program definition?
        if parsers.is_start_of_program_block(cs):
            if co:
                _run_non_prog_lines(co)
                co = []

        co.append(c)

        # Are we ending a program definition?
        if cs == 'end':
            _run_prog_lines(co)
            co = []

    if co:
        _run_non_prog_lines(co)

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_execute(self, code, silent, store_history=True, user_expressions=None,
               allow_stdin=False):
    if not self.stata_ready:
        self.init_stata()
    Cell(self, code).run()
    self.shell.execution_count += 1
    return {
        'status': 'ok',
        'execution_count': self.execution_count,
        'payload': [],
        'user_expressions': {},
        }

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()